In [1]:
from agential.cog.agent.reflexion import ReflexionCoTAgent, ReflexionReActAgent
from agential.cog.prompts.agent.reflexion import (
    REFLEXION_COT_INSTRUCTION_HOTPOTQA,
    HOTPOTQA_FEWSHOT_EXAMPLES_REFLEXION_COT_REFLECT,
    REFLEXION_COT_REFLECT_INSTRUCTION_HOTPOTQA,
    REFLEXION_REACT_INSTRUCTION_HOTPOTQA,
    REFLEXION_REACT_REFLECT_INSTRUCTION_HOTPOTQA,
    HOTPOTQA_FEWSHOT_EXAMPLES_REFLEXION_REACT_REFLECT,
)
from agential.cog.prompts.benchmark.hotpotqa import (
    HOTPOTQA_FEWSHOT_EXAMPLES_COT,
    HOTPOTQA_FEWSHOT_EXAMPLES_REACT
)

from langchain_openai import ChatOpenAI

import dotenv
dotenv.load_dotenv()

import warnings
warnings.filterwarnings('ignore')

llm = ChatOpenAI()

In [ ]:
question = "VIVA Media AG changed it's name in 2004. What does their new acronym stand for?"
key = "Gesellschaft mit beschränkter Haftung"


agent = ReflexionCoTAgent(
    llm=llm,
    mode={"qa": "hotpotqa"},
)
out = agent.generate(
    question=question, 
    key=key, 
    examples=HOTPOTQA_FEWSHOT_EXAMPLES_COT,
    prompt=REFLEXION_COT_INSTRUCTION_HOTPOTQA, 
    reflect_examples=HOTPOTQA_FEWSHOT_EXAMPLES_REFLEXION_COT_REFLECT,
    reflect_prompt=REFLEXION_COT_REFLECT_INSTRUCTION_HOTPOTQA,
    reflection_strategy="reflexion",
    additional_keys={},
    reflection_additional_keys={},
    max_trials=3,
    patience=3,
    reset=True
)

In [ ]:
out

In [4]:
question = "VIVA Media AG changed it's name in 2004. What does their new acronym stand for?"
key = "Gesellschaft mit beschränkter Haftung"


agent = ReflexionReActAgent(
    llm=llm,
    mode={"qa": "hotpotqa"},
)
out = agent.generate(
    question=question, 
    key=key, 
    examples=HOTPOTQA_FEWSHOT_EXAMPLES_REACT,
    prompt=REFLEXION_REACT_INSTRUCTION_HOTPOTQA, 
    reflect_examples=HOTPOTQA_FEWSHOT_EXAMPLES_REFLEXION_REACT_REFLECT,
    reflect_prompt=REFLEXION_REACT_REFLECT_INSTRUCTION_HOTPOTQA,
    reflection_strategy="last_attempt_and_reflexion",
    additional_keys={},
    reflection_additional_keys={},
    max_steps=6,
    max_trials=3,
    patience=3,
    reset=True
)

<PROMPT AGENT=====================================================>
Solve a question answering task with interleaving Thought, Action, Observation steps. Thought can reason about the current situation, and Action can be three types: 
(1) Search[entity], which searches the exact entity on Wikipedia and returns the first paragraph if it exists. If not, it will return some similar entities to search.
(2) Lookup[keyword], which returns the next sentence containing keyword in the last passage successfully found by Search.
(3) Finish[answer], which returns the answer and finishes the task.
You have a maximum of 6 steps.

Here are some examples:
Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
Thought 1: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.
Action 1: Search[Colorado orogeny]
Observation 1: The Colorado orogeny was 